The purpose of this notebook is to demonstrate the functionality of the `VARReduce` model from sktime. We will compare its performance with the traditional `VAR` model to highlight their similar behavior under default settings. Additionally, we will showcase how `VARReduce` can be leveraged for regularization in scenarios involving large time series datasets, offering enhanced model performance and robustness.

Vector Autoregression (VAR) is a powerful statistical tool used to capture the linear interdependencies among multiple time series. Despite its complex applications, the core mechanism of fitting a VAR model is fundamentally an extension of linear regression. This article explores how we can reframe the VAR fitting process into a more flexible framework by leveraging different types of regressors, including those that provide regularization.

### Reframing VAR Model Fitting: From Linear Regression to Regularized Approaches

Vector Autoregression (VAR) is a powerful statistical tool used to capture the linear interdependencies among multiple time series. Despite its complex applications, the core mechanism of fitting a VAR model is fundamentally an extension of linear regression. This article explores how we can reframe the VAR fitting process into a more flexible framework by leveraging different types of regressors, including those that provide regularization.

#### Understanding Linear Regression

Linear regression models the relationship between a dependent variable \( y \) and one or more independent variables \( X \). The model is expressed as:

\[ y_t = \beta_0 + \beta_1 x_{1t} + \beta_2 x_{2t} + \cdots + \beta_p x_{pt} + \epsilon_t \]

Here:
- \( y_t \) is the dependent variable at time \( t \).
- \( x_{it} \) are the independent variables at time \( t \).
- \( \beta_0, \beta_1, \ldots, \beta_p \) are the coefficients.
- \( \epsilon_t \) is the error term.

The goal is to estimate the coefficients \( \beta \) that minimize the sum of squared errors.

#### The VAR Model

A VAR model generalizes this concept to multiple time series. For \( k \) time series variables \( y_{1,t}, y_{2,t}, \ldots, y_{k,t} \), a VAR model of order \( p \) (VAR(p)) is:

\[ 
\begin{aligned}
y_{1,t} &= c_1 + \sum_{j=1}^{k} \sum_{l=1}^{p} \beta_{1j,l} y_{j,t-l} + \epsilon_{1,t} \\
y_{2,t} &= c_2 + \sum_{j=1}^{k} \sum_{l=1}^{p} \beta_{2j,l} y_{j,t-l} + \epsilon_{2,t} \\
&\vdots \\
y_{k,t} &= c_k + \sum_{j=1}^{k} \sum_{l=1}^{p} \beta_{kj,l} y_{j,t-l} + \epsilon_{k,t} 
\end{aligned}
\]

Here:
- \( y_{i,t} \) is the value of the \( i \)-th time series at time \( t \).
- \( c_i \) are the intercepts.
- \( \beta_{ij,l} \) are the coefficients for the \( j \)-th variable's \( l \)-lag effect on the \( i \)-th variable.
- \( \epsilon_{i,t} \) are the error terms.

### Reframing VAR Fitting: Tabularization Followed by Regression

To simplify the VAR fitting process, we can transform the time series data into a tabular (matrix) format and then apply regression techniques.

#### Step 1: Tabularization (Matrix Formulation)

- **Time Series to Tabular Data**: Convert each time series into a matrix where each row represents a time point and the columns represent the lagged values of all time series variables.
- **Example**: For a VAR(2) model with two time series \( y_1 \) and \( y_2 \), the matrix might look like this:

  \[
  \begin{pmatrix}
  y_{1,t-1} & y_{2,t-1} & y_{1,t-2} & y_{2,t-2} \\
  y_{1,t} & y_{2,t} & y_{1,t-1} & y_{2,t-1} \\
  y_{1,t+1} & y_{2,t+1} & y_{1,t} & y_{2,t} \\
  \vdots & \vdots & \vdots & \vdots
  \end{pmatrix}
  \]

- **Response Variables**: Create response vectors for each time series variable representing the value at the current time point.
- **Example**: For \( y_1 \), the response vector might be:

  \[
  \begin{pmatrix}
  y_{1,t} \\
  y_{1,t+1} \\
  y_{1,t+2} \\
  \vdots
  \end{pmatrix}
  \]

#### Step 2: Regression

Once we have our data in a tabular format, we can use various regression techniques to estimate the coefficients.

### Alternative Regressors for Regularized VAR

1. **Ridge Regression (L2 Regularization)**:
   - Adds a penalty proportional to the sum of the squares of the coefficients.
   - Helps prevent overfitting by shrinking the coefficients.

     \[
     \min_{\beta} \sum_{i=1}^{n} (y_i - X_i \beta)^2 + \lambda \sum_{j=1}^{p} \beta_j^2
     \]

2. **Lasso Regression (L1 Regularization)**:
   - Adds a penalty proportional to the sum of the absolute values of the coefficients.
   - Can drive some coefficients to zero, effectively performing variable selection.

     \[
     \min_{\beta} \sum_{i=1}^{n} (y_i - X_i \beta)^2 + \lambda \sum_{j=1}^{p} |\beta_j|
     \]

3. **Elastic Net Regression**:
   - Combines L1 and L2 regularization.
   - Balances between ridge and lasso by tuning a mixing parameter.

     \[
     \min_{\beta} \sum_{i=1}^{n} (y_i - X_i \beta)^2 + \lambda_1 \sum_{j=1}^{p} |\beta_j| + \lambda_2 \sum_{j=1}^{p} \beta_j^2
     \]

### Steps to Implement Regularized VAR

1. **Tabularize the Data**:
   - Construct the lagged matrix \( X \) and response vectors \( Y_1, Y_2, \ldots, Y_k \) for each time series.

2. **Choose a Regularized Regressor**:
   - Use Ridge, Lasso, or Elastic Net regression to fit each response vector on the lagged matrix.

3. **Estimate Coefficients**:
   - Solve the regression problem for each response vector to get the coefficients.

4. **Construct the VAR Model**:
   - Use the estimated coefficients to form the VAR model, which now includes regularization.

### Example Code

Here’s a brief example using Python and scikit-learn to demonstrate how you might implement regularized VAR:

```python
from sklearn.linear_model import Ridge, Lasso, ElasticNet
import numpy as np

# Assuming X is the lagged matrix and Y is the response vector for a single time series

# Example data (replace with actual lagged data)
X = np.random.rand(100, 4)  # 100 time points, 4 lagged variables
Y = np.random.rand(100)     # 100 time points

# Ridge Regression
ridge = Ridge(alpha=1.0)
ridge.fit(X, Y)
ridge_coefficients = ridge.coef_

# Lasso Regression
lasso = Lasso(alpha=0.1)
lasso.fit(X, Y)
lasso_coefficients = lasso.coef_

# Elastic Net Regression
elastic_net = ElasticNet(alpha=1.0, l1_ratio=0.5)
elastic_net.fit(X, Y)
elastic_net_coefficients = elastic_net.coef_

print("Ridge Coefficients:", ridge_coefficients)
print("Lasso Coefficients:", lasso_coefficients)
print("Elastic Net Coefficients:", elastic_net_coefficients)
```

### Conclusion

By reframing the problem of fitting a VAR model as tabularization followed by regression, we open up a range of possibilities for using different types of regressors. This approach not only simplifies the implementation but also allows for the incorporation of regularization techniques like Ridge, Lasso, and Elastic Net. These regularized approaches can enhance the flexibility and robustness of the VAR model, making it more capable of handling complex datasets and preventing overfitting.

In [5]:
import numpy as np
import pandas as pd

from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error
from sktime.forecasting.var_reduce import VARReduce
from statsmodels.tsa.api import VAR

from sklearn.preprocessing import StandardScaler
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.compose import ForecastingPipeline
from sktime.transformations.series.adapt import TabularToSeriesAdaptor
from sklearn.linear_model import Ridge

# Suppress all warnings
import warnings
warnings.filterwarnings("ignore")


In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data Generation

To perform the comparison between `VAR` and `VARReduce`, we first generate a small synthetic multivariate time series dataset. 

We create three time series, each with 100 observations, and store them in a DataFrame. 

We then split the data into training and testing sets, reserving the last 5 observations for testing.

In [7]:
np.random.seed(42)
n_obs = 100
time_series_1 = np.random.randn(n_obs) + 1000
time_series_2 = np.random.randn(n_obs) + 100
time_series_3 = np.random.randn(n_obs) + 10

df = pd.DataFrame(
    {
        "ts1": time_series_1,
        "ts2": time_series_2,
        "ts3": time_series_3,
    }
)

# Split the data into training and testing sets
df_train, df_test = temporal_train_test_split(df, test_size=5)

# Define the forecasting horizon
fh = ForecastingHorizon(df_test.index, is_relative=False)

# Model Introduction

The `VARReduce` model applies Vector Autoregression (VAR) with optional regularization. 
It operates by converting multivariate time series data into a tabular format suitable for regression models. By default, `VARReduce` uses `scikit-learn`'s `LinearRegression` to perform the fitting and behaves like a traditional VAR model. However, users can specify other `scikit-learn` compatible regressors to introduce regularization, enhancing performance for large datasets or when multicollinearity is present.

Here's how the `VARReduce` model works step-by-step:

1. Data Preparation: The time series data is transformed into a format where each row contains lagged values of the series, turning it into a tabular dataset.
2. Model Fitting: The model uses the specified regressor to fit the lagged values as predictors and the current values as the response variable.
3. Forecasting: The fitted model is used to forecast future values based on the most recent observations.

To demonstrate the intermediate DataFrame generated during the data preparation step, we'll take a peep at the internal `prepare_var_data` method from the `VARReduce` class.

In [18]:
# Define lags
LAGS = 2

# Instantiate and fit the custom VARReduce model
varreduce_model = VARReduce(lags=LAGS, regressor = Ridge(alpha = 100))  # no regressor passed, by default, LinearRegressor() will be used
varreduce_model.fit(df_train)

# Fit the VAR model comparison
var_model = VAR(endog=df_train)

# Fit the VAR model using statsmodels for comparison
var_model = VAR(df_train)
results =var_model.fit(LAGS)

In [19]:
# Define lags
LAGS = 2

# Instantiate and fit the custom VARReduce model
varreduce_model = VARReduce(lags=LAGS, regressor = Ridge(alpha = 0.2))  # no regressor passed, by default, LinearRegressor() will be used
varreduce_model.fit(df_train)

# Fit the VAR model comparison
var_model = VAR(endog=df_train)

# Fit the VAR model using statsmodels for comparison
var_model = VAR(df_train)
results =var_model.fit(LAGS)


We observe that internally, `VARReduce` transforms the provided training data into a tabular format suitable for regression. Specifically:

1. **Predictors (X)**: The predictors consist of lagged values of the time series, with the number of lags specified by the `lags` parameter. In the displayed DataFrame, each column represents a lagged value for one of the time series (e.g., `ts1_lag1`, `ts2_lag2`).

2. **Target Variables (y)**: The target variables are the current, unlagged values of the time series, shown in columns such as `ts1_lag0`, `ts2_lag0`, and `ts3_lag0`.



In [20]:
X, y = varreduce_model.prepare_var_data(df_train, return_as_ndarray=False)
X.head()

,ts1_lag1,ts2_lag1,ts3_lag1,ts1_lag2,ts2_lag2,ts3_lag2
2,999.861736,99.579355,10.560785,1000.496714,98.584629,10.357787
3,1000.647689,99.657285,11.083051,999.861736,99.579355,10.560785
4,1001.523030,99.197723,11.053802,1000.647689,99.657285,11.083051
5,999.765847,99.838714,8.622331,1001.523030,99.197723,11.053802
6,999.765863,100.404051,9.062175,999.765847,99.838714,8.622331


In [21]:
y.head()

,ts1_lag0,ts2_lag0,ts3_lag0
2,1000.647689,99.657285,11.083051
3,1001.523030,99.197723,11.053802
4,999.765847,99.838714,8.622331
5,999.765863,100.404051,9.062175
6,1001.579213,101.886186,10.515035


# Coefficient Comparison

Once the data is prepared in this tabular format, `VARReduce` trains a regressor on it. By default, `LinearRegression` from scikit-learn is used as the regressor. Under these default conditions, the `VARReduce` model behaves just like a traditional VAR model. This means that the fitted coefficients and the resulting forecasts will be identical to those of a VAR model, as both are essentially performing linear regression on the same lagged data.

This equivalence is demonstrated below:

In [22]:
# Compare coefficients
print("Coefficients from VARReduce model:")
print(varreduce_model.coefficients_)
print("\nCoefficients from statsmodels VAR:")
print(results.coefs)

Coefficients from VARReduce model:
[[[-0.04361311 -0.1432403  -0.0286208 ]
  [-0.02681481 -0.12353761 -0.09681436]
  [-0.02880361 -0.12741472 -0.11312612]]

 [[-0.04871809 -0.05343262 -0.01240621]
  [ 0.11886914 -0.05718469 -0.131875  ]
  [ 0.09278917  0.09723644 -0.03899424]]]

Coefficients from statsmodels VAR:
[[[-0.04379616 -0.14362626 -0.02868124]
  [-0.02691002 -0.12388731 -0.09702357]
  [-0.02884349 -0.12771343 -0.11331339]]

 [[-0.0488787  -0.0536462  -0.01244942]
  [ 0.11919954 -0.05735081 -0.13221751]
  [ 0.0930699   0.09744015 -0.0391535 ]]]


# Forecast Comparison

Given that the two models have essentially identical fitted parameters, it is unsurprising that their forecasts are also essentially identical:

In [23]:
# Generating forecasts from both models
df_pred_varreduce = varreduce_model.predict(fh=fh)
df_pred_statsmodels = results.forecast(df_train.values[-LAGS:], steps=len(fh))

In [24]:
df_pred_varreduce

,ts1,ts2,ts3
1,999.953792,99.960219,9.816463
2,999.918117,100.005744,10.045984
3,999.907816,100.103436,10.047713
4,999.890680,100.054364,10.027533
5,999.894295,100.055801,10.045038


In [25]:
df_pred_statsmodels

array([[999.95405968,  99.95999686,   9.81591952],
       [999.9181998 , 100.00566178,  10.04602297],
       [999.9078646 , 100.10367569,  10.04778239],
       [999.89062779, 100.05432332,  10.0274662 ],
       [999.89427888, 100.05578669,  10.04508813]])

In [26]:
# Compare predictions using numpy's allclose function
epsilon = 1e-8
predictions_are_close = np.allclose(df_pred_varreduce, df_pred_statsmodels, atol=epsilon)

print("Are the predictions from VARReduce and sktime VAR close? ", predictions_are_close)


Are the predictions from VARReduce and sktime VAR close?  False


# Example 2: Real World Macreconomic Data

The power of `VARReduce` becomes more evident when dealing with real-world datasets containing numerous time series. An example is the `macrodata` dataset, which contains 14 variables. This dataset is loaded below:

In [27]:
import statsmodels.api as sm

# Load the macrodata dataset using statsmodels
dataset = sm.datasets.macrodata.load_pandas()
df = dataset.data
df['year'] = df['year'].astype(int)
df['quarter'] = df['quarter'].astype(int)

# Create a datetime column
df['date'] = pd.to_datetime(df['year'].astype(str) + 'Q' + df['quarter'].astype(str))

# Set the datetime column as the index
df.set_index('date', inplace=True)

# Convert the datetime index to a period index with quarterly frequency
df.index = df.index.to_period('Q')

# Drop the original year and quarter columns if no longer needed
df.drop(columns=['year', 'quarter'], inplace=True)

df.head()

,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
date,,,,,,,,,,,,
1959Q1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1959Q2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
1959Q3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
1959Q4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
1960Q1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [28]:
# Split the data into training and testing sets
df_train, df_test = temporal_train_test_split(df, test_size=24)
fh = ForecastingHorizon(df_test.index, is_relative=False)

Training a traditional VAR model on this dataset with a lag of 3 means we have to fit a substantial number of parameters. Specifically, for each variable, the model needs to estimate coefficients for the current and three previous values of all 14 variables, plus an intercept. This results in a total of \( (14 \times 14 \times 3) + 14 = 602 \) parameters. Given that the number of data points is relatively small, this high number of parameters makes the model prone to overfitting, where it captures noise in the training data rather than the underlying patterns.

Overfitting leads to poor generalization to new, unseen data, resulting in inaccurate forecasts. To mitigate this, regularization techniques can be applied, which is where `VARReduce` shines. By selecting a regressor with built-in regularization properties, such as Ridge regression (L2 regularization) or Lasso regression (L1 regularization), we can introduce penalties on the size of the coefficients. This effectively controls the complexity of the model, preventing overfitting, enhancing stability, and improving the forecast accuracy.

For instance, using Ridge regression within `VARReduce` helps to manage multicollinearity and shrink the less important coefficients towards zero, making the model more interpretable and reliable. This ability to integrate advanced regression techniques allows `VARReduce` to leverage the flexibility and robustness of scikit-learn regressors, providing a powerful tool for time series forecasting in complex, high-dimensional datasets.

In [29]:


# Define lags
LAGS = 3

# Create the pipeline for VARReduce
scaler = TabularToSeriesAdaptor(StandardScaler())
varreduce_model = VARReduce(lags=LAGS, regressor = Ridge(alpha = 10))
pipeline_varreduce = ForecastingPipeline(steps=[("scaler", scaler), ("forecaster", varreduce_model)])

# Fit the pipeline
pipeline_varreduce.fit(df_train)
df_pred_varreduce = pipeline_varreduce.predict(fh=fh)

# Create the pipeline for VAR using sktime
pipeline_var = ForecastingPipeline(steps=[("scaler", scaler), ("forecaster", VAR(maxlags=LAGS))])

# Fit the pipeline
pipeline_var.fit(df_train)
df_pred_var = pipeline_var.predict(fh=fh)

RuntimeError: Cannot clone object VARReduce(lags=3, regressor=Ridge(alpha=10)), as the constructor either does not set or modifies parameter regressor

In [ ]:
# Calculate performance metrics for VARReduce
mape_varreduce = mean_absolute_percentage_error(df_test, df_pred_varreduce)
print(f"MAPE for VARReduce model: {mape_varreduce:.2f}")

# Calculate performance metrics for statsmodels VAR
mape_var = mean_absolute_percentage_error(df_test, df_pred_var)
print(f"MAPE for statsmodels VAR model: {mape_var:.2f}")